## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-18-47-05 +0000,nypost,DSA politician’s aide admits to pushing subord...,https://nypost.com/2025/09/25/us-news/dsa-poli...
1,2025-09-25-18-46-22 +0000,nyt,Trump Wades Into Bailout Politics in Offering ...,https://www.nytimes.com/2025/09/25/us/politics...
2,2025-09-25-18-43-43 +0000,bbc,PM defends mayor against 'ridiculous' Trump cl...,https://www.bbc.com/news/articles/c147vd511djo...
3,2025-09-25-18-43-06 +0000,nypost,Horror moment unstoppable wall of water from m...,https://nypost.com/2025/09/25/world-news/super...
4,2025-09-25-18-42-18 +0000,nypost,Fatal shooting of 13-year-old NYC boy likely o...,https://nypost.com/2025/09/25/us-news/fatal-sh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,45
327,ice,17
175,president,17
36,shooting,13
104,kimmel,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
129,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...,118
34,2025-09-25-17-49-10 +0000,nypost,Trump calls out Turkish president for buying R...,https://nypost.com/2025/09/25/us-news/trump-ca...,100
78,2025-09-25-16-25-34 +0000,nypost,Disney gears up for court battle with Trump af...,https://nypost.com/2025/09/25/business/disney-...,100
45,2025-09-25-17-19-02 +0000,nypost,Trump warns of violence against ‘radical left’...,https://nypost.com/2025/09/25/us-news/trump-wa...,92
320,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,91


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
129,118,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...
90,78,2025-09-25-15-57-45 +0000,nypost,Joshua Jahn’s family claims Dallas shooter was...,https://nypost.com/2025/09/25/us-news/joshua-j...
80,60,2025-09-25-16-17-15 +0000,nypost,Over 100 ex-ABC journalists urge Disney’s Bob ...,https://nypost.com/2025/09/25/business/over-10...
142,58,2025-09-25-13-09-54 +0000,bbc,Former French president Sarkozy given five-yea...,https://www.bbc.com/news/articles/cp98kepmj9lo...
64,53,2025-09-25-16-58-18 +0000,cbc,Liberal government frees Canada Post to end ho...,https://www.cbc.ca/news/politics/canada-post-t...
248,48,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
308,46,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
54,43,2025-09-25-17-10-56 +0000,nypost,Chinese drone experts have been working with R...,https://nypost.com/2025/09/25/world-news/chine...
94,39,2025-09-25-15-52-04 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...
81,37,2025-09-25-16-16-13 +0000,nypost,"Starbucks slashing 900 jobs, closing hundreds ...",https://nypost.com/2025/09/25/business/starbuc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
